# CDFS SWIRE master catalogue
## Preparation of COMBO data

Classifying Objects by Medium-Band Observations - a spectrophotometric 17-filter survey (COMBO-17). COMBO catalogue: the catalogue comes from `dmu0_COMBO-17`.

In the catalogue, we keep:

- The identifier (it's unique in the catalogue);
- The position;
- The stellarity;
- The total magnitude (aperture magnitudes are not provided.

We don't know when the maps have been observed. We will use the year of the reference paper.

In [1]:
from herschelhelp_internal import git_version
print("This notebook was run with herschelhelp_internal version: \n{}".format(git_version()))

This notebook was run with herschelhelp_internal version: 
f5975da (Wed Jul 12 18:03:06 2017 +0100)


In [2]:
%matplotlib inline
#%config InlineBackend.figure_format = 'svg'

import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))

from collections import OrderedDict
import os

from astropy import units as u
from astropy import constants as const
from astropy.coordinates import SkyCoord
from astropy.table import Column, Table
import numpy as np

from herschelhelp_internal.flagging import  gaia_flag_column
from herschelhelp_internal.masterlist import nb_astcor_diag_plot, remove_duplicates
from herschelhelp_internal.utils import astrometric_correction, mag_to_flux, flux_to_mag

/Users/rs548/anaconda/envs/herschelhelp_internal/lib/python3.6/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [3]:
OUT_DIR =  os.environ.get('TMP_DIR', "./data_tmp")
try:
    os.makedirs(OUT_DIR)
except FileExistsError:
    pass

RA_COL = "combo_ra"
DEC_COL = "combo_dec"

## I - Column selection

In [10]:
imported_columns = OrderedDict({
    'Seq':'combo_id', 
    'ra':'combo_ra', 
    'dec':'combo_dec',  
    #'dl':'combo_dl',
    'stellarity':'combo_stellarity',
    #'Rmag':'m_combo_r',    #The first bands are fitted absolute magnitudes
    #'e_Rmag':'merr_combo_r',  
    #'UjMag':'m_combo_uj',   
    #'e_UjMag':'merr_combo_uj', 
    #'BjMag':'m_combo_bj',   
    #'e_BjMag':'merr_combo_bj',
    #'VjMag':'m_combo_vj',   
    #'e_VjMag':'merr_combo_vj',
    #'usMag':'m_combo_us',   
    #'e_usMag':'merr_combo_us',
    #'gsMag':'m_combo_gs',   
    #'e_gsMag':'merr_combo_gs',
    #'rsMag':'m_combo_rs',   
    #'e_rsMag':'merr_combo_rs',
    #'UbMag':'m_combo_ub',   
    #'e_UbMag':'merr_combo_ub',
    #'BbMag':'m_combo_bb',   
    #'e_BbMag':'merr_combo_bb',
    #'VbMag':'m_combo_vb',   
    #'e_VbMag':'merr_combo_vb',
    #'S280Mag':'m_combo_s280', 
    #'e_S280Mag':'merr_combo_s280',
    ##'S145Mag':'m_combo_s145', 
    #'e_S145Mag':'merr_combo_s145',
    'W420F_E':'f_combo_420', #The following values are given as fluxes
    'e_W420F_E':'ferr_combo_420',
    'W462F_E':'f_combo_462',
    'e_W462F_E':'ferr_combo_462',
    'W485F_D':'f_combo_485',
    'e_W485F_D':'ferr_combo_485',
    'W518F_E':'f_combo_518',
    'e_W518F_E':'ferr_combo_518',
    'W571F_S':'f_combo_571',   #Combined flux from two runs
    'e_W571F_S':'ferr_combo_571',
    'W604F_E':'f_combo_604',
    'e_W604F_E':'ferr_combo_604',
    'W646F_D':'f_combo_646',
    'e_W646F_D':'ferr_combo_646',
    'W696F_E':'f_combo_696',
    'e_W696F_E':'ferr_combo_696',
    'W753F_E':'f_combo_753',
    'e_W753F_E':'ferr_combo_753',
    'W815F_S':'f_combo_815',
    'e_W815F_S':'ferr_combo_815',
    'W856F_D':'f_combo_856',
    'e_W856F_D':'ferr_combo_856',
    'W914F_D':'f_combo_914', #Two runs but no combined - taking first
    'e_W914F_D':'ferr_combo_914',
    'UF_S':'f_combo_u',
    'e_UF_S':'ferr_combo_u',
    'BF_S':'f_combo_b',
    'e_BF_S':'ferr_combo_b',
    'VF_D':'f_combo_v',
    'e_VF_D':'ferr_combo_v',
    'RF_S':'f_combo_r',
    'e_RF_S':'ferr_combo_r',
    'IF_D':'f_combo_i',
    'e_IF_D':'ferr_combo_i'
    })


 

catalogue = Table.read("../../dmu0/dmu0_COMBO-17/data/table3.fits")[list(imported_columns)]
for column in imported_columns:
    catalogue[column].name = imported_columns[column]

epoch = 2000 #table says 1999 to 2001

# Clean table metadata
catalogue.meta = None

## Unit conversion
The flux is presented in $\textrm{photons} .\textrm{s}^{-1} .  \textrm{m}^{-2}  .\textrm{nm}^{-1}$. We wish to convert these to micro Jansky; $10^{-32} \textrm{ W} . \textrm{m}^{-2}  . \textrm{Hz}^{-1} $.

To convert $\textrm{photons} . \textrm{s}^{-1} $ to $\textrm{W}$ we must multiply by the average photon energy $h c / \lambda$. We presume that the COMBO mid point wavelength was used.

To convert $\textrm{nm}^{-1}$ to $\textrm{Hz}^{-1}$ we must differenciate:

$c = \nu \lambda $

$\nu = c / \lambda$

$\frac{d \nu}{d \lambda} = - c /\lambda^{2}$

$d \lambda = -  (\lambda^{2} / c )\times d \nu$



The net result of this is to multiply by $\lambda^2 / c$.

Combining these two unit conversions leads to overall multiplying by $ h \lambda$:

$(\lambda^2 / c ) \times (h c / \lambda) = h \lambda$

$\frac{\lambda^2 }{ c } \times \frac{h c }{ \lambda} = h \lambda$

In [49]:
#Example conversion from photon s^-1 m^-2 nm^-1 to Jy
flux_lambda = 0.0064185 * (u.m **-2) * (u.s ** -1) *( u.nm ** -1)

wavelength = 420 * u.nm
flux_nu = flux_lambda * const.h *  wavelength
print('flux_lambda:', flux_lambda)
#print('f_lambda:', flux_lambda.decompose())
print('flux_nu:', flux_nu)
print('flux_nu in Jy:',flux_nu.to(u.Jy))


flux_lambda: 0.0064185 1 / (m2 nm s)
flux_nu: 1.78623595647189e-33 J / m2
flux_nu in Jy: 1.7862359564718896e-07 Jy


In [11]:
#Replace 0.0 with NaN values
for col in catalogue.colnames:
    if col.startswith('m'): # | col.endswith('ra') | col.endswith('dec'):
        catalogue[col][np.where(catalogue[col] == 0.0)] = np.nan

# Add flux and band-flag columns
for col in catalogue.colnames:
    if col.startswith('f_'):
        catalogue[col] = catalogue[col] 
        errcol = "ferr{}".format(col[1:])
        magnitude, error = flux_to_mag(np.array(catalogue[col])*1.e-6, np.array(catalogue[errcol])*1.e-6)

        # Fluxes are added in µJy
        catalogue.add_column(Column(magnitude, name="m{}".format(col[1:])))
        catalogue.add_column(Column(error, name="m{}".format(errcol[1:])))
        
        #We add NAN filled aperture columns because no aperture fluxes are present
        nancol = np.zeros(len(catalogue))
        nancol.fill(np.nan)
        catalogue.add_column(Column(nancol, 
                                    name="m_ap{}".format(col[1:])))
        catalogue.add_column(Column(nancol, 
                                    name="merr_ap{}".format(col[1:])))
        catalogue.add_column(Column(nancol, 
                                    name="f_ap{}".format(col[1:])))
        catalogue.add_column(Column(nancol, 
                                    name="ferr_ap{}".format(col[1:])))
        
        # 
        
        # Band-flag column
        catalogue.add_column(Column(np.zeros(len(catalogue), dtype=bool), name="flag{}".format(col[1:])))

/Users/rs548/GitHub/herschelhelp_internal/herschelhelp_internal/utils.py:76: RuntimeWarning: invalid value encountered in log10
  magnitudes = 2.5 * (23 - np.log10(fluxes)) - 48.6


In [12]:
catalogue[:10].show_in_notebook()

idx,combo_id,combo_ra,combo_dec,combo_stellarity,f_combo_420,ferr_combo_420,f_combo_462,ferr_combo_462,f_combo_485,ferr_combo_485,f_combo_518,ferr_combo_518,f_combo_571,ferr_combo_571,f_combo_604,ferr_combo_604,f_combo_646,ferr_combo_646,f_combo_696,ferr_combo_696,f_combo_753,ferr_combo_753,f_combo_815,ferr_combo_815,f_combo_856,ferr_combo_856,f_combo_914,ferr_combo_914,f_combo_u,ferr_combo_u,f_combo_b,ferr_combo_b,f_combo_v,ferr_combo_v,f_combo_r,ferr_combo_r,f_combo_i,ferr_combo_i,m_combo_420,merr_combo_420,m_ap_combo_420,merr_ap_combo_420,f_ap_combo_420,ferr_ap_combo_420,flag_combo_420,m_combo_462,merr_combo_462,m_ap_combo_462,merr_ap_combo_462,f_ap_combo_462,ferr_ap_combo_462,flag_combo_462,m_combo_485,merr_combo_485,m_ap_combo_485,merr_ap_combo_485,f_ap_combo_485,ferr_ap_combo_485,flag_combo_485,m_combo_518,merr_combo_518,m_ap_combo_518,merr_ap_combo_518,f_ap_combo_518,ferr_ap_combo_518,flag_combo_518,m_combo_571,merr_combo_571,m_ap_combo_571,merr_ap_combo_571,f_ap_combo_571,ferr_ap_combo_571,flag_combo_571,m_combo_604,merr_combo_604,m_ap_combo_604,merr_ap_combo_604,f_ap_combo_604,ferr_ap_combo_604,flag_combo_604,m_combo_646,merr_combo_646,m_ap_combo_646,merr_ap_combo_646,f_ap_combo_646,ferr_ap_combo_646,flag_combo_646,m_combo_696,merr_combo_696,m_ap_combo_696,merr_ap_combo_696,f_ap_combo_696,ferr_ap_combo_696,flag_combo_696,m_combo_753,merr_combo_753,m_ap_combo_753,merr_ap_combo_753,f_ap_combo_753,ferr_ap_combo_753,flag_combo_753,m_combo_815,merr_combo_815,m_ap_combo_815,merr_ap_combo_815,f_ap_combo_815,ferr_ap_combo_815,flag_combo_815,m_combo_856,merr_combo_856,m_ap_combo_856,merr_ap_combo_856,f_ap_combo_856,ferr_ap_combo_856,flag_combo_856,m_combo_914,merr_combo_914,m_ap_combo_914,merr_ap_combo_914,f_ap_combo_914,ferr_ap_combo_914,flag_combo_914,m_combo_u,merr_combo_u,m_ap_combo_u,merr_ap_combo_u,f_ap_combo_u,ferr_ap_combo_u,flag_combo_u,m_combo_b,merr_combo_b,m_ap_combo_b,merr_ap_combo_b,f_ap_combo_b,ferr_ap_combo_b,flag_combo_b,m_combo_v,merr_combo_v,m_ap_combo_v,merr_ap_combo_v,f_ap_combo_v,ferr_ap_combo_v,flag_combo_v,m_combo_r,merr_combo_r,m_ap_combo_r,merr_ap_combo_r,f_ap_combo_r,ferr_ap_combo_r,flag_combo_r,m_combo_i,merr_combo_i,m_ap_combo_i,merr_ap_combo_i,f_ap_combo_i,ferr_ap_combo_i,flag_combo_i
,,,,,phot/m^2/s/nm,phot/m^2/s/nm,phot/m^2/s/nm,phot/m^2/s/nm,phot/m^2/s/nm,phot/m^2/s/nm,phot/m^2/s/nm,phot/m^2/s/nm,phot/m^2/s/nm,phot/m^2/s/nm,phot/m^2/s/nm,phot/m^2/s/nm,phot/m^2/s/nm,phot/m^2/s/nm,phot/m^2/s/nm,phot/m^2/s/nm,phot/m^2/s/nm,phot/m^2/s/nm,phot/m^2/s/nm,phot/m^2/s/nm,phot/m^2/s/nm,phot/m^2/s/nm,phot/m^2/s/nm,phot/m^2/s/nm,phot/m^2/s/nm,phot/m^2/s/nm,phot/m^2/s/nm,phot/m^2/s/nm,phot/m^2/s/nm,phot/m^2/s/nm,phot/m^2/s/nm,phot/m^2/s/nm,phot/m^2/s/nm,phot/m^2/s/nm,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,7864,52.9043509407,-28.004407161,0.704,0.0064185,0.00297409,0.00875974,0.00258908,0.00924363,0.00384037,0.010762,0.00314495,0.00418278,0.00290015,0.00520915,0.00312953,0.0103215,0.00948629,0.00915815,0.00275962,0.0168907,0.00568383,0.00671985,0.00305952,0.0188677,0.00513084,0.0120776,0.00749415,0.00727563,0.00203166,0.00538465,0.00134767,0.00324916,0.00163189,0.00515757,0.000448774,0.00463687,0.00384278,29.3814,0.503089,nan,nan,nan,nan,False,29.0438,0.320907,nan,nan,nan,nan,False,28.9854,0.451081,nan,nan,nan,nan,False,28.8203,0.317282,nan,nan,nan,nan,False,29.8463,0.7528,nan,nan,nan,nan,False,29.6081,0.652284,nan,nan,nan,nan,False,28.8656,0.997878,nan,nan,nan,nan,False,28.9955,0.327164,nan,nan,nan,nan,False,28.3309,0.365356,nan,nan,nan,nan,False,29.3316,0.494331,nan,nan,nan,nan,False,28.2107,0.295253,nan,nan,nan,nan,False,28.695,0.673697,nan,nan,nan,nan,False,29.2453,0.303183,nan,nan,nan,nan,False,29.5721,0.271739,nan,nan,nan,nan,False,30.1206,0.545313,nan,nan,nan,nan,False,29.6189,0.0944727,nan,nan,nan,nan,False,29.7344,0.899797,nan,nan,nan,nan,False
1,7867,52.9248830635,-28.0045354786,0.659,0.00463079,0.0032505,0.00338548,0.

## II - Removal of duplicated sources

We remove duplicated objects from the input catalogues.

In [ ]:
SORT_COLS = ['merr_combo_r',  
     'merr_combo_uj', 
     'merr_combo_bj',
     'merr_combo_vj',
     'merr_combo_us',
     'merr_combo_gs',
     'merr_combo_rs',
     'merr_combo_ub',
     'merr_combo_bb',
     'merr_combo_vb',
     'merr_combo_s280',
     'merr_combo_s145']
FLAG_NAME = 'combo_flag_cleaned'

nb_orig_sources = len(catalogue)

catalogue = remove_duplicates(catalogue, RA_COL, DEC_COL, sort_col=SORT_COLS,flag_name=FLAG_NAME)

nb_sources = len(catalogue)

print("The initial catalogue had {} sources.".format(nb_orig_sources))
print("The cleaned catalogue has {} sources ({} removed).".format(nb_sources, nb_orig_sources - nb_sources))
print("The cleaned catalogue has {} sources flagged as having been cleaned".format(np.sum(catalogue[FLAG_NAME])))

## III - Astrometry correction

We match the astrometry to the Gaia one. We limit the Gaia catalogue to sources with a g band flux between the 30th and the 70th percentile. Some quick tests show that this give the lower dispersion in the results.

In [ ]:
gaia = Table.read("../../dmu0/dmu0_GAIA/data/GAIA_CDFS-SWIRE.fits")
gaia_coords = SkyCoord(gaia['ra'], gaia['dec'])

In [ ]:
nb_astcor_diag_plot(catalogue[RA_COL], catalogue[DEC_COL], 
                    gaia_coords.ra, gaia_coords.dec)

In [ ]:
delta_ra, delta_dec =  astrometric_correction(
    SkyCoord(catalogue[RA_COL], catalogue[DEC_COL]),
    gaia_coords
)

print("RA correction: {}".format(delta_ra))
print("Dec correction: {}".format(delta_dec))

In [ ]:
catalogue[RA_COL] +=  delta_ra.to(u.deg)
catalogue[DEC_COL] += delta_dec.to(u.deg)

In [ ]:
nb_astcor_diag_plot(catalogue[RA_COL], catalogue[DEC_COL], 
                    gaia_coords.ra, gaia_coords.dec)

## IV - Flagging Gaia objects

In [ ]:
catalogue.add_column(
    gaia_flag_column(SkyCoord(catalogue[RA_COL], catalogue[DEC_COL]), epoch, gaia)
)

In [ ]:
GAIA_FLAG_NAME = "combo_flag_gaia"

catalogue['flag_gaia'].name = GAIA_FLAG_NAME
print("{} sources flagged.".format(np.sum(catalogue[GAIA_FLAG_NAME] > 0)))

## V - Flagging objects near bright stars

# VI - Saving to disk

In [ ]:
catalogue.write("{}/COMBO.fits".format(OUT_DIR), overwrite=True)